In [1]:
%matplotlib widget
%matplotlib widget

import ipywidgets as widgets
from IPython.display import clear_output

import jupyros as jr
import rospy

from std_msgs.msg import String
from a3_code.srv import Coordinate_xy    #used in case 1

from IPython.display import display

from geometry_msgs.msg import Twist, Vector3    #for cmd_vel topic

from jupyros import ros3d
import os


import matplotlib.pyplot as plt
import tf
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation


reached_task = []

In [2]:
"""
#useless to be deleted
#directly moved in an other position
def odom_visualization():
    print("starting visualization")
    vis = Visualiser()
    sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
    ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
    plt.show(block=True)
"""    

'\n#useless to be deleted\n#directly moved in an other position\ndef odom_visualization():\n    print("starting visualization")\n    vis = Visualiser()\n    sub = rospy.Subscriber(\'/odom\', Odometry, vis.odom_callback)\n    ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)\n    plt.show(block=True)\n'

In [3]:
def set_cmd_vel(direction):
    #direction is considerable as a matrix 3x3
    #up-left    up     up-right
    #left       pause  right
    #down-left  down   down-right
    
    #initialize a Twist object for the publisher
    init = Vector3(0, 0, 0)
    vel = Twist( init, init)
    
    lin_vel = 0.5
    ang_vel = 0.5
    
    if direction == 1:
        vel.linear.x = lin_vel
        vel.angular.z = ang_vel
    
    elif direction == 2:
        vel.linear.x = lin_vel
        
    elif direction == 3:
        vel.linear.x = lin_vel
        vel.angular.z = -ang_vel
        
    elif direction == 4:
        vel.angular.z = ang_vel
        
#    elif direction == 5:
        #do nothing
    elif direction == 6:
        vel.angular.z = -ang_vel
    elif direction == 7:
        vel.linear.x = -lin_vel
        vel.angular.z = ang_vel
    elif direction == 8:
        vel.linear.x = -lin_vel
    elif direction == 9:
        vel.linear.x = -lin_vel
        vel.angular.z = -ang_vel
    
    
    #initialize the publisher
    pub = rospy.Publisher('remap_cmd_vel',Twist, queue_size=10)
    #pubblic on topic remap_cmd_vel to the robot
    pub.publish(vel)
    

In [4]:
def drive_interface():
    


#    display(buttonUP, buttonDOWN, buttonLEFT, buttonRIGHT, buttonPAUSE, toggle)
    display(arrow_box)
    display(toggle)
    display(button4)
    
    
    
    buttonUP.on_click(on_button_clicked_UP)
    buttonDOWN.on_click(on_button_clicked_DOWN)
    buttonLEFT.on_click(on_button_clicked_LEFT)
    buttonRIGHT.on_click(on_button_clicked_RIGHT)
    buttonPAUSE.on_click(on_button_clicked_PAUSE)
    
    buttonUP_LEFT.on_click(on_button_clicked_UP_LEFT)
    buttonUP_RIGHT.on_click(on_button_clicked_UP_RIGHT)
    buttonDOWN_LEFT.on_click(on_button_clicked_DOWN_LEFT)
    buttonDOWN_RIGHT.on_click(on_button_clicked_DOWN_RIGHT)
    toggle.observe(toggle_handler, names='value')
    
    button4.on_click(on_button_clicked_4)
    
    #display the ros3d map
    display_map()

In [5]:
def display_map():
    v = ros3d.Viewer()
    rc = ros3d.ROSConnection(url="ws://localhost:9090")
    tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')

    laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
    map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client, continuous=True)
    path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc,
    tf_client=tf_client)
    urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client,
    path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))
    g = ros3d.GridModel()

    #v.objects = [g, laser_view, map_view, path, urdf]
    v.objects = [laser_view, map_view, urdf]

    display(v)

In [6]:
#call back functions for the button click event
def on_button_clicked_1(b):
    with output:
        print("Button 1 clicked.")
        button1.disabled=True
        button2.disabled=True
        
        display(box_x, box_y, button3, button4)
          
        button3.on_click(on_button_clicked_3)
        button4.on_click(on_button_clicked_4)
        
        
        
def on_button_clicked_2(b):
    with output:
        #open drive interface
        
        button2.disabled=True
        button1.disabled=True
        
        drive_interface()
        
        
        
#submit the action for reaching a coordinate and wait for a response       
def on_button_clicked_3(b):
    with output:
        #print("Button 3 clicked.")
        
        button3.disabled=True
        print(box_x.value, box_y.value)
        
        rospy.wait_for_service('coordinate_xy')
        coordinate_xy = rospy.ServiceProxy('coordinate_xy', Coordinate_xy)
        rt = coordinate_xy(box_x.value , box_y.value)
        if rt.return_ == 1:
            print("target reached successfully!")
            reached_task.append('reached')
        else:
            print("target not reached")
            reached_task.append('not reached')
        display(button4)
        button4.on_click(on_button_clicked_4)
        

def on_button_clicked_4(b):
    with output:
        clear_output()
        button1.disabled=False
        button2.disabled=False
        button3.disabled=False
        

#handle all the different buttons for manually drive the robot
#calling the associated set_cmd_vel function with the correct parameter
def on_button_clicked_UP(b):
    set_cmd_vel(2)
def on_button_clicked_DOWN(b):
    set_cmd_vel(8)
def on_button_clicked_LEFT(b):
    set_cmd_vel(4)
def on_button_clicked_RIGHT(b):
    set_cmd_vel(6)
def on_button_clicked_PAUSE(b):
    set_cmd_vel(5)

def on_button_clicked_UP_LEFT(b):
    set_cmd_vel(1)
def on_button_clicked_UP_RIGHT(b):
    set_cmd_vel(3)
def on_button_clicked_DOWN_LEFT(b):
    set_cmd_vel(7)
def on_button_clicked_DOWN_RIGHT(b):
    set_cmd_vel(9)
    
def toggle_handler(b):
    rospy.set_param("/collision_avoidance", toggle.value)

In [7]:
#%matplotlib widget


#used for display the plot of the robot posizion using topic /pose
class Visualiser:
    
    index = 0
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []
    
    def plot_init(self):
        self.ax.set_xlim( -10, 10)
        self.ax.set_ylim( -10, 10)
        return self.ln
    
    def odom_callback(self, msg):
        
        self.index = self.index+1
        
        if self.index == 250:
            if len(self.y_data) < 500:
                self.y_data.append(msg.pose.pose.position.y)
            else:
                self.y_data.pop(0)
                self.y_data.append(msg.pose.pose.position.y)

            if len(self.x_data) < 500:
                self.x_data.append(msg.pose.pose.position.x)
            else:
                self.x_data.pop(0)
                self.x_data.append(msg.pose.pose.position.x)
                
            self.index = 0
        
            
    
    def update_plot(self, frame):
        #print("updateing")
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

    
    
#used for display the laserScan sensor output
#use of a polar plot for a simpler interpretation of the data
#note that the robot heading angle is moved from from to 90deg for semplicity of rappresentation
class Visualiser2:
    
    def __init__(self):
        self.fig, self.ax = plt.subplots( subplot_kw={'projection': 'polar'} )
        self.ln, = plt.plot([], [], 'r.' )
        self.theta_data = np.arange( 0, np.pi, np.pi/720.0)   #[rad]
        self.d_data = []
    
    def plot_init(self):
        #self.ax.set_xlim( -10, 10)
        #self.ax.set_ylim( -10, 10)
        self.ax.set_rmax(15)
        return self.ln
    
        
    def laserScan_callback(self, msg):
            self.d_data = msg.ranges
    
    def update_plot(self, frame):
        #print("updateing")
        self.ln.set_data( self.theta_data, self.d_data )
        return self.ln




#intialization of the ros node 'jupyter_user_interface'
rospy.init_node('jupyter_user_interface')


#create buttons
button1 = widgets.Button(description="reach coordinate")
button2 = widgets.Button(description="drive robot")
button3 = widgets.Button(description="start moving")
button4 = widgets.Button(description="back to menu")

#movement interface buttons
buttonUP = widgets.Button(
#    description="up",
    icon="arrow-up")

buttonDOWN = widgets.Button(
#    description="down",
    icon="arrow-down")

buttonLEFT = widgets.Button(
#    description="left",
    icon="arrow-left")

buttonRIGHT = widgets.Button(
#    description="right",
    icon="arrow-right")

buttonPAUSE = widgets.Button(
#    description="pause",
    icon="pause")


buttonUP_LEFT = widgets.Button(
    description = "up left",
#    icon="pause"
)

buttonUP_RIGHT = widgets.Button(
    description = "up right",
#    icon="pause"
)

buttonDOWN_LEFT = widgets.Button(
    description = "down left",
#    icon="pause"
)

buttonDOWN_RIGHT = widgets.Button(
    description = "down right",
#    icon="pause"
)

#widget used for displaing the control button in a simple configuration
arrow_box = widgets.HBox(
    [widgets.VBox([buttonUP_LEFT,  buttonLEFT,  buttonDOWN_LEFT]),
    widgets.VBox( [buttonUP,       buttonPAUSE, buttonDOWN]),
    widgets.VBox( [buttonUP_RIGHT, buttonRIGHT, buttonDOWN_RIGHT])]
    )

#toggle widget for activate/deactivate the collision avoidance option during the manual navigation of the robot
toggle = widgets.Checkbox(
    value = False,
    description='active collision avoidance'
)

#boxes for entering the coordinate of the taget to reach
box_x = widgets.FloatText(description="x coordinate: ")
box_y = widgets.FloatText(description="y coordinate: ")



#create an output cell
output = widgets.Output()

#display both button and output cell
#display main menu of the GUI
display(button1, button2, output)

#moved in the GUI for the robot driving
#display_map()

#listener for buttons 1,2
button1.on_click(on_button_clicked_1)
button2.on_click(on_button_clicked_2)


#visualization of the first plot --> position of the robot
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit=True)
plt.show(block=True)

#visualization of the second plot --> data from LaserScan sensor
vis2 = Visualiser2()
sub2 = rospy.Subscriber('/scan', LaserScan, vis2.laserScan_callback)
ani2 = FuncAnimation(vis2.fig, vis2.update_plot, init_func=vis2.plot_init, blit=True)
plt.show(block=True)    



#histogram
fig, ax = plt.subplots()
_, _, ln, = ax.hist([])

def init():
    return ln,

def update(frame):
    ax.hist(reached_task, bins=3, color='green')
    return ln,

ani3 = FuncAnimation(fig, update, init_func=init, blit=True)
plt.show()

Button(description='reach coordinate', style=ButtonStyle())

Button(description='drive robot', style=ButtonStyle())

Output()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …